In [1]:
import numpy as np
import os
import json
import src.utils as utils

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix

from src.ml_statistical_features import load_data_as_dataframe, get_all_scores, get_rf_grid_params, get_lda_grid_params, get_dt_grid_params, get_mlp_grid_params, get_linear_svc_grid_params, eval_classifier_paper
from src.notebook_md_utils import get_md_data_distribution_string, get_md_mean_accuracy_grid, get_md_test_accuracy_grid, get_md_confusion_matrix_grid

from IPython.display import Markdown, display

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# Daten laden

In [3]:
data = load_data_as_dataframe(segment_length=10, overlap_amount=0, hr_threshold=15)
x = data.iloc[:, 0:13]
y = data['informative_hr']
patient_id = data['patient_id']


data['abs_error'] = np.abs(data['bcg_hr']-data['ecg_hr'])
data['rel_error'] = 100/data['ecg_hr'] * data['abs_error']

# Splitten der Daten in G1 und G2

## Labelverteilung

### Im Paper:
58% informativ

42% nicht-informativ


In [4]:
distribution = np.bincount(y)

display(Markdown("### Vorliegende Daten"))
display(Markdown("%i %s informativ" % (round(100/len(y)*distribution[1]), '%')))
display(Markdown("%i %s nicht-informativ" % (round(100/len(y)*distribution[0]), '%')))

### Vorliegende Daten

50 % informativ

50 % nicht-informativ


## Gruppenverteilung
Segmente zufällig in 2 Gruppen unterteilt
### Im Paper


|   | informativ | nicht-informativ | gesamt    |   
|:--|:--------- :|:----------------:|:---------:|
| G1 |1296 (62%)  | 789 (38%)        | 2085 (57%)|
| G2 |813 (53%)   | 733 (47%)        | 1546 (43%)|

In [5]:
x_g1, x_g2, y_g1, y_g2 = train_test_split(x, y, test_size=0.43, random_state=1, stratify=y)

display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_data_distribution_string(y_g1, y_g2)))

### Vorliegende Daten

|   | informativ | nicht-informativ | gesamt    |
|:--|:--------- :|:----------------:|:---------:|
|G1 | 24402 (50%)  | 24452 (50%)| 48854 (57%)|
|G2 | 18409 (50%)  | 18446 (50%)| 36855 (43%)|


## TODO: Coverage + Mean Error

## 10-Fold cross validation mean accuracy for G1 and G2

### Im Paper

(RF: ntrees=50, SVM: rbf kernel, NN: 50 hidden neuron)

|    | RF    | SVM   | NN    | LDA   | DT    |
|:---|:-----:|:-----:|:-----:|:-----:|:-----:|
| G1 | 98.13 | 93.38 | 91.61 | 89.26 | 97.51 |
| G2 | 92.30 | 90.49 | 85.89 | 79.37 | 89.39 |

In [7]:
paths_paper = ['LDA_paper_hr15', 'DT_paper_hr15', 'RF_paper_hr15', 'MLP_paper_hr15', 'SVC_paper_hr10']
scores = get_all_scores(reconstruct=False, paths=paths_paper)


display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_mean_accuracy_grid(scores)))

### Vorliegende Daten

| | LDA| DT| RF| MLP| SVM| 
|:--|:--:|:--:|:--:|:--:|:--:|
 | G1 |  58.56  |  57.69  |  63.52  |  61.33  |  61.00  | 
 | G2 |  58.52  |  57.15  |  63.34  |  60.97  |  60.38  | 

## Accuracy results for testing G2 vs. G1 (Exp1) and testing G1 vs. G2 (Exp2)

### Im Paper

|      | RF    | SVM   | NN    | LDA   | DT    |
|:-----|:-----:|:-----:|:-----:|:-----:|:-----:|
| Exp1 | 100   | 94.44 | 92.28 | 89.40 | 97.51 |
| Exp2 | 97.99 | 97.46 | 87.10 | 90.26 | 97.41 |
| Mean | 98.995| 95.95 | 89.69 | 89.83 | 98.41 |

In [8]:
display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_test_accuracy_grid(scores)))

### Vorliegende Daten

| | LDA| DT| RF| MLP| SVM| 
|:--|:--:|:--:|:--:|:--:|:--:|
 | Exp1 |  58.46  |  57.76  |  63.32  |  61.13  |  61.72  | 
 | Exp2 |  58.59  |  57.28  |  63.15  |  61.37  |  61.12  | 
 | Mean |  58.52 |  57.52 |  63.23 |  61.25 |  61.42 | 

## Confusion matrix of random forest for Exp2

### Im Paper

|             |                  | Actual        |                  |
|:------------|:----------------:|:-------------:|:----------------:|
|             |                  | informativ    | nicht-informativ |
|             |                  |               |                  |
|**Predicted**| informativ       | 1270          | 26               |
|             | nicht-informativ | 18            | 771              |

In [9]:
rf, _ = get_rf_grid_params()
_, _, _, _, _, y_pred_rf, y_true = eval_classifier_paper(x, y, patient_id, clf=rf, grid_folder_name='RF_paper_hr15')
conf_mat = confusion_matrix(y_true, y_pred_rf)

display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

### Vorliegende Daten

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 14839 | 8475 |
|| nicht-informativ | 9563 | 15977 |

### Andere Klassifikatoren

In [10]:
lda, _ = get_lda_grid_params()
_, _, _, _, _, y_pred_lda, y_true = eval_classifier_paper(x, y, patient_id, clf=lda, grid_folder_name='LDA_paper_hr15')
conf_mat = confusion_matrix(y_true, y_pred_lda)

display(Markdown("#### LDA"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

#### LDA

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 16666 | 12550 |
|| nicht-informativ | 7736 | 11902 |

In [14]:
dt, _ = get_dt_grid_params()
_, _, _, _, _, y_pred_dt, y_true = eval_classifier_paper(x, y, patient_id, clf=dt, grid_folder_name='DT_paper_hr15')
conf_mat = confusion_matrix(y_true, y_pred_dt)

display(Markdown("#### Decision Tree"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

#### Decision Tree

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 14118 | 10461 |
|| nicht-informativ | 10284 | 13991 |

In [ ]:
mlp, _ = get_mlp_grid_params()
_, _, _, _, _, y_pred_mlp, y_true = eval_classifier_paper(x, y, patient_id, clf=mlp, grid_folder_name='MLP_paper_hr15')
conf_mat = confusion_matrix(y_true, y_pred_mlp)

display(Markdown("#### MLP"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

In [ ]:
svm, _ = get_linear_svc_grid_params()
_, _, _, _, _, y_pred_svm, y_true = eval_classifier_paper(x, y, patient_id, clf=svm, grid_folder_name='SVC_paper_hr10')
conf_mat = confusion_matrix(y_true, y_pred_svm)

display(Markdown("#### SVM"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

# Fehler nach RF Klassifizierung

In [13]:
test_set = data.loc[y_true.index]
informativ_annotiert = test_set[test_set['informative_hr']]
non_informativ_annotiert = test_set[test_set['informative_hr'] == False]
non_informativ_annotiert = non_informativ_annotiert.replace([np.inf, -np.inf], np.nan)
informativ_rf = test_set[y_pred_rf]
informativ_rf = informativ_rf.replace([np.inf, -np.inf], np.nan)
non_informative_rf = test_set[y_pred_rf == False]
non_informative_rf = non_informative_rf.replace([np.inf, -np.inf], np.nan)

print("Relativer Fehler der Herzrate bei als informativ annotierten Segmenten %.2f %s" % (informativ_annotiert['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als informativ annotierten Segmenten %.2f %s" % (informativ_annotiert['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten %i" % (informativ_annotiert['abs_error'].isna().sum()))
print("Relativer Fehler der Herzrate bei als nicht informativ annotierten Segmenten %.2f %s" % (non_informativ_annotiert['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als nicht informativ annotierten Segmenten %.2f %s" % (non_informativ_annotiert['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als nicht informativ annotierten Segmenten %i" % (non_informativ_annotiert['abs_error'].isna().sum()))

print("\n")

print("Relativer Fehler der Herzrate bei als informativ vorausgesagten Segmenten %.2f %s" % (informativ_rf['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als informativ vorausgesagten Segmenten %.2f %s" % (informativ_rf['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten %i" % (informativ_rf['abs_error'].isna().sum()))
print("Relativer Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten %.2f %s" % (non_informative_rf['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten %.2f %s" % (non_informative_rf['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als nicht informativ vorausgesagten Segmenten %i" % (non_informative_rf['abs_error'].isna().sum()))

Relativer Fehler der Herzrate bei als informativ annotierten Segmenten 6.75 %
Absoluter Fehler der Herzrate bei als informativ annotierten Segmenten 4.58  bpm
Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten 0
Relativer Fehler der Herzrate bei als nicht informativ annotierten Segmenten 29.17 %
Absoluter Fehler der Herzrate bei als nicht informativ annotierten Segmenten 20.92  bpm
Anzahl Segmente mit NaN Fehler bei als nicht informativ annotierten Segmenten 67


Relativer Fehler der Herzrate bei als informativ vorausgesagten Segmenten 13.89 %
Absoluter Fehler der Herzrate bei als informativ vorausgesagten Segmenten 9.40  bpm
Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten 0
Relativer Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten 21.67 %
Absoluter Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten 15.81  bpm
Anzahl Segmente mit NaN Fehler bei als nicht informativ vorausgesagten Segmenten 67
